In [1]:
import requests
import pandas as pd
import os
import re
from pandas.io.json import json_normalize
import json

爬取歌单名称：华语评论1W＋

歌单共计2000余首华语评论1000+的歌曲，涵盖了较老的歌和较新的歌，包罗万象

歌单作者id：133511246

歌单id：728958078

API:/playlist/track/all?id=728958078&limit=50&offset=0

首先先获取歌单里面所有歌曲的id

In [2]:
url = "http://127.0.0.1:3000/playlist/track/all?id=728958078&limit=10&offset=0"
response = requests.request("GET", url).json()

In [3]:
response

{'songs': [{'name': '危险派对',
   'id': 1877996649,
   'pst': 0,
   't': 0,
   'ar': [{'id': 12236125, 'name': '王以太', 'tns': [], 'alias': []},
    {'id': 1172023, 'name': '刘至佳', 'tns': [], 'alias': []}],
   'alia': [],
   'pop': 100,
   'st': 0,
   'rt': '',
   'fee': 8,
   'v': 27,
   'crbt': None,
   'cf': '',
   'al': {'id': 133202644,
    'name': '危险派对',
    'picUrl': 'https://p1.music.126.net/GpnLproqUUyc4xmYKpRFcQ==/109951166516282895.jpg',
    'tns': [],
    'pic_str': '109951166516282895',
    'pic': 109951166516282900},
   'dt': 180571,
   'h': {'br': 320000, 'fid': 0, 'size': 7225005, 'vd': -57704, 'sr': 48000},
   'm': {'br': 192000, 'fid': 0, 'size': 4335021, 'vd': -55129, 'sr': 48000},
   'l': {'br': 128000, 'fid': 0, 'size': 2890029, 'vd': -53450, 'sr': 48000},
   'sq': {'br': 977692, 'fid': 0, 'size': 22067908, 'vd': -57699, 'sr': 48000},
   'hr': {'br': 1747706,
    'fid': 0,
    'size': 39448232,
    'vd': -57699,
    'sr': 48000},
   'a': None,
   'cd': '01',
   'no': 1,

In [4]:
def GetSongs(offset):
    url_ = "http://127.0.0.1:3000/playlist/track/all?id=728958078&limit=1000&offset=" + str(offset)
    data_ = requests.request("GET", url_).json()
    df_ = pd.json_normalize(data_['songs'])
    return df_['id']

In [5]:
list1 = GetSongs(0).tolist()
list2 = GetSongs(1000).tolist()
list3 = GetSongs(2000).tolist()
list1.extend(list2)
list1.extend(list3)

In [6]:
len(list1)

2213

In [25]:
len(list1[1000:2000])

1000

经过复核歌单确实有2213首歌曲。接下来按照表单`list1`来爬取这中间每一首歌的热度最高的五条评论。

API:/comment/new?type=0&id=1877996649&sortType=2&pageSize=5

In [11]:
# 获取歌曲前五条热评
def GetComments(id):
    url_ = "http://127.0.0.1:3000/comment/new?type=0&id=" + str(id) + "&sortType=2&pageSize=5"
    data_ = requests.request("GET", url_).json()
    COLUMNS = ["user.userId","commentId", "user.nickname", "content", "timeStr", "likedCount", 'user.isHug', 'replyCount', 'user.authStatus']
    df_ = pd.json_normalize(data_['data']["comments"])
    return df_[COLUMNS]

In [12]:
def get_all_comments(song_list):
    ans = GetComments(song_list[0])
    for i in range(1,len(song_list)):
        new_df = GetComments(song_list[i])
        ans = pd.concat([ans, new_df])
    return ans

In [13]:
hot_comments = get_all_comments(list1)

In [32]:
hot_comments.to_csv("hot_comments_auth.csv",encoding='utf_8_sig')

In [15]:
hot_comments.head(50)

,user.userId,commentId,user.nickname,content,timeStr,likedCount,user.isHug,replyCount,user.authStatus
0,3291743001,5428603245,玛卡巴卡在阿巴阿巴,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,2021-10-01,95720,False,1688,0
1,267627817,5427142964,皮崽-承蒙厚爱,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],2021-09-29,68632,False,468,1
2,428956762,5419339232,保久伽壹,“右手托住你的心跳” 怎么……托😬,2021-09-17,61614,False,2304,0
3,1984646294,5446242912,阿泽潮音,你说你大姨妈走了 让我晚上去你家 宝 守灵这活哥真干不了啊,2021-10-27,44296,False,628,0
4,287711061,5420069620,等X的新歌,托住你的心跳这句可太高情商了,2021-09-18,37329,False,221,0
0,47774029,9074483,胖了一整个盛夏,开头金润吉的那几句简直好听炸！！！！！！！！！！\n,2015-01-20,27464,False,134,0
1,67404460,128604071,小白不会很白,阿里郎这个名字起的俗，不然能火[大哭]是这样的吗,2016-03-12,18959,False,209,0
2,128843665,339992770,若凤求凰,有人问我思念到极致是什么感觉。 我曾经发了句晚安给她，一晚上醒来六、七次看手机信息。 就是...,2017-03-26,16671,False,322,0
3,122700987,135827338,你我皆凡人_,今天2016年03月30号。这首歌会火！,2016-03-30,13079,False,1603,0
4,4022472656,5627762154,圈钱公主,如果你刷到我 愿你嫁给爱情,2022-04-03,9039,False,312,0


In [16]:
# 获取歌曲前15条最新评论
def GetNewComments(id):
    url_ = "http://127.0.0.1:3000/comment/new?type=0&id=" + str(id) + "&sortType=3&pageSize=15"
    data_ = requests.request("GET", url_).json()
    COLUMNS = ["user.userId","commentId", "user.nickname", "content", "timeStr", "likedCount", 'user.isHug', 'replyCount']
    df_ = pd.json_normalize(data_['data']["comments"])
    return df_[COLUMNS]

In [17]:
def get_all_new_comments(song_list):
    ans = GetNewComments(song_list[0])
    for i in range(1,len(song_list)):
        new_df = GetNewComments(song_list[i])
        ans = pd.concat([ans, new_df])
    return ans

In [26]:
new_comments_1 = get_all_new_comments(list1[0:1000])

In [27]:
new_comments_2 = get_all_new_comments(list1[1000:])

In [29]:
new_comments = pd.concat([new_comments_1, new_comments_2])

In [30]:
new_comments

,user.userId,commentId,user.nickname,content,timeStr,likedCount,user.isHug,replyCount
0,1894471710,5905868958,银雾1906,6,3分钟前,0,False,0
1,6413620741,5905772200,摆烂的旅行者啊,有没有看过散兵和流浪者一起跳的。越看越涩,09:32,0,False,0
2,3811137102,5905769848,理想超sky,让我想起了优拉跳舞的画面[18],09:25,1,False,0
3,8558486705,5905764073,许雨-1,是我们没有挺过去 别怪时间 更别说不适合.,09:11,0,False,0
4,1690241045,5905747468,就把他打死吧,音乐合伙人,08:20,0,False,0
...,...,...,...,...,...,...,...,...
10,8371314927,5902779771,_L-yyyyyy,困住我的是内心，是思念，是徘徊，是你说你从来不爱我。,6天前,0,False,0
11,8371314927,5902779735,_L-yyyyyy,没有在为情所困 只是觉得身边的人都不诚,6天前,0,False,0
12,8371314927,5902776250,_L-yyyyyy,我试图挽留过，但没用。,6天前,0,False,0
13,8371314927,5902775604,_L-yyyyyy,我试图挽留过，但没用。,6天前,0,False,0


In [31]:
new_comments.to_csv("new_comments.csv",encoding='utf_8_sig')